# Analysis

In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from bidict import bidict
import sys
import os
from itertools import product
if "/u/solimanz/development/career_prediction_gpu/" not in sys.path:
    sys.path.append("/u/solimanz/development/career_prediction_gpu/")

In [2]:
df_path = "/data/rali7/Tmp/solimanz/data/pickles/excerpt-2017-02-20_reduced.pkl"
base_path = "/data/rali7/Tmp/solimanz/data/"
top550_path = os.path.join(base_path, "datasets", "top550")
reduced7000_path = os.path.join(base_path, "datasets", "reduced7000")
predictions_path = os.path.join(base_path, "model_predictions")

In [3]:
with open(os.path.join(top550_path, "jobid", "data.json"), "r") as f:
    data = json.load(f)

In [6]:
title_id = bidict(data["title_to_id"])

In [15]:
def groupby_length(seq_lengths):
    d = dict()    
    for i in range(2, seq_lengths.max()):
        d[i] = np.where(seq_lengths == i)
        
    return d

In [4]:
def next_batch(path):
    batch = 0
    name = os.path.basename(path)
    while os.path.exists(os.path.join(path, f"{name}_batch_{batch}.npy")):
        with open(os.path.join(path, f"{name}_batch_{batch}.npy"), "rb") as f:
            matrix = np.load(f)
        batch += 1
        yield matrix

In [5]:
targets_path = os.path.join(predictions_path, "top550", "jobid", "targets")
inputs_path = os.path.join(predictions_path, "top550", "jobid", "inputs")
seq_lengths_path = os.path.join(predictions_path, "top550", "jobid", "seq_lengths")

bow_predictions_path = os.path.join(predictions_path, "top550", "bow", "predictions")
jobid_predictions_path = os.path.join(predictions_path, "top550", "jobid", "predictions")
title_emb_predictions_path = os.path.join(predictions_path, "top550", "title_emb", "predictions")

In [6]:
targets_gen = next_batch(targets_path)
inputs_gen = next_batch(inputs_path)
seq_length_gen = next_batch(seq_lengths_path)

jobid_preds_gen = next_batch(jobid_predictions_path)
bow_preds_gen = next_batch(bow_predictions_path)
title_emb_preds_gen = next_batch(title_emb_predictions_path)

In [7]:
targets = []
inputs = []
seq_lengths = []

jobid_preds = []
bow_preds = []
title_emb_preds = []

for i, t, sl, jobid, bow, title_emb in zip(inputs_gen,
                                           targets_gen,
                                           seq_length_gen,
                                           jobid_preds_gen,
                                           bow_preds_gen,
                                           title_emb_preds_gen):
    inputs.append(i)
    targets.append(t)
    seq_lengths.append(sl)
    
    jobid_preds.append(jobid.reshape((jobid.shape[1],jobid.shape[2], jobid.shape[3])))
    bow_preds.append(bow.reshape((bow.shape[1],bow.shape[2], bow.shape[3])))
    title_emb_preds.append(title_emb.reshape((title_emb.shape[1],title_emb.shape[2], title_emb.shape[3])))
    
    

In [8]:
inputs = np.concatenate(inputs, axis=0)
targets = np.concatenate(targets, axis=0)
seq_lengths = np.concatenate(seq_lengths, axis=0)

jobid_preds = np.concatenate(jobid_preds, axis=0)
bow_preds = np.concatenate(bow_preds, axis=0)
title_emb_preds = np.concatenate(title_emb_preds, axis=0)

In [13]:
np.where(seq_lengths == 2)[0]

array([    0,     1,     2, ..., 24076, 24077, 24078])

In [24]:
jobid_preds[indices].shape

(5813, 32, 550)

In [89]:
correct = np.equal(np.argmax(targets, 2), np.argmax(jobid_preds, 2)).astype(np.int16)

In [16]:
jobid_preds.shape

(24080, 32, 550)

In [44]:
z = np.zeros((24080, 1, 550))

In [49]:
np.concatenate([bow_preds, z], axis=1).shape

(24080, 32, 550)